![SolidQ](https://www.solidq.com/wp-content/uploads/2015/06/Logo-SolidQ-Web.gif)

# Estadística Descriptiva para variables continuas

In [ ]:
import pyodbc
import pandas as pd 
import matplotlib.pyplot as plt 

In [ ]:
driver = 'DRIVER={ODBC Driver 17 for SQL Server}'
server = 'tcp:solidqdemo.database.windows.net' 
database = 'AdventureWorksDW2017' 
usme =usuarioa
 
passwr"P0lici@2020"
' 
vTargetMail = "SELECT CustomerKey, Age, YearlyIncome,CommuteDistance,BikeBuyer FROM vTargetMail"
#Abrimos la conexion
cnxn = pyodbc.codriver +'erver};SERVER='+server+';DATABASE='+database+';UID=ername+';PWD='+ password)

In [ ]:
TM = pd.read_sql(vTargetMail,cnxn)

In [ ]:
#Estadística desriptiva básica
TM['Age'].describe()

In [ ]:
TM['Age'].mean()
TM['Age'].median()

In [ ]:
#Midiendo la dispersión
#La forma más sencilla, es midiendo el rango
TM['Age'].max()-TM['Age'].min()

In [ ]:
#utilizando el IQR(Inter-quartile range)
TM['Age'].quantile(0.75) - TM['Age'].quantile(0.25)

In [ ]:
#Coeficiente de variación
TM['Age'].std() / TM['Age'].mean()

In [ ]:
#Skweness mide la asimetría en la distribución de probabilidad. Si es positiva la disitribución tiene un "long tail" a la derecha de la media
TM['Age'].skew()

In [ ]:
#Si la distribución tiene "long tails" en ambas partes - Kurtosis. Si es negativa indica que los tails son cortos
TM['Age'].kurt()

# Variables discretas

In [ ]:
import pyodbc
import pandas as pd 
import matplotlib.pyplot as plt 

In [ ]:
server = 'tcp:solidqdemo.database.windows.net' 
database = 'AdventureWorksDW2017' 
username = 'adminsql' 
password = 'Vit@e2019' 
vTargetMail = "SELECT CustomerKey, TotalChildren, NumberChildrenAthome,Gender,HouseOwnerFlag,NumberCarsOwned, MaritalStatus, Age, YearlyIncome, \
    SpanishEducation AS Education, SpanishOccupation AS Occupation FROM vTargetMail"
#Abrimos la conexion
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

In [ ]:
df=pd.read_sql(vTargetMail, cnxn)

In [ ]:
df['Education'].value_counts()

In [ ]:
#Definirlo como ordinal
df['Education'] = df['Education'].astype('category')
df['Education'].cat.reorder_categories(["Educación secundaria (en curso)","Educación secundaria","Estudios universitarios (en curso)","Licenciatura","Estudios de postgrado"],inplace=True)
df['EducationInt'] = df['Education'].cat.codes
df['Education'].value_counts().sort_index()

In [ ]:
edu=df['Education'].value_counts().sort_index()
ax=edu.plot(kind='bar',color=('b'),fontsize=14, legend=False,use_index=True,rot=1)
ax.set_xlabel('Education',fontsize=16)
ax.set_ylabel('Count',fontsize=16)
plt.show()

# Mostrar asociaciones gráficamente

In [ ]:
TM1= df.head(200)

In [ ]:
plt.scatter(TM1['Age'],TM1['YearlyIncome'])
plt.show()

# Datos Categóricos

In [ ]:
import pandas as pd
df = pd.DataFrame([
  ['verde', 'M', 10.1, 'clase1'],
  ['rojo', 'L', 13.5, 'clase2'],
   ['azul', 'XL', 15.3, 'clase1']])
df.columns = ['color', 'talla', 'precio', 'clase']
df
#característica nominal, el color, una ordinal la talla y una numérica, el precio

# Mapear Características ordinales
Para asegurarnos de que el algoritmo de aprendizaje interpreta las características ordinales correctamente, necesitamos convertir las cadenas de caracteres categóricas en enteros. Desafortunadamente aún no hemos visto una función que nos permita hacerlo automáticamente, derivando el orden correcto de las etiquetas de nuestra caractarística, por lo que vamos a definirlo manualmente. En el siguiente ejemplo, asumimos que sabemos la diferencia numérica entre las características, por ejemplo: XL = L +1 = M + 2:

In [ ]:
size_mapping = {'XL': 3,'L': 2,'M': 1}
df['talla'] = df['talla'].map(size_mapping)
df

más tarde, podemos simplemente definir un diccionario de mapeo inverso inv_size_mapping = {v: k for k, v in size_mapping.items()} 
que puede utilizarse con el método map en la característica transformada, similar a lo que hemos hecho anteriormente

In [ ]:
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['talla'].map(inv_size_mapping)

# Codificando etiquetas de clases
Muchas librerías de ML requieren que las etiquetas de las clases estén codificadas como valores enteros. Aunque muchos estimadores de clasificación en scikit-learn convierten las etiquetas en enteros internamente, está considerado como una buena práctica, proporcionanrlas directamente como un array de enteros para evitar problemas. Para codificar las etiquetas, podemos utilizar una aproximación similar al mapeo de características ordinales que hemos visto. Tenemos que recordar que las etiquetas de clases no son ordinales, por lo que no importa que valor entero les asignemos. Podemos simplemente enumerarlas comenzando en 0

In [ ]:
import numpy as np
class_mapping = {label:idx for idx,label in enumerate(np.unique(df['clase']))}
class_mapping

In [ ]:
df['clase'] = df['clase'].map(class_mapping)
df

In [ ]:
#Vuelta atrás
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['clase'] = df['clase'].map(inv_class_mapping)
df

Podemos utilizar la clase LabelEncoder del paquete scikit-learn para hacer este trabajo directamente, pero esto lo veremos en detalle en el siguiente tema

from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['clase'].values)
y


# Discretizando variables continuas

In [ ]:
#Con el mismo ancho. Abiertas por laizquierda
TM1['Edad'] = pd.cut(TM1['Age'],5)
TM1['Edad'].value_counts()

In [ ]:
#Si queremos hacerlos de igual alto
TM1['Edad2'] = pd.qcut(TM1['Age'],5)
TM1['Edad2'].value_counts(sort=False)

In [ ]:
#Personalizado
custombins = [16,22,29,39,54,88]
TM1['Edad3']=pd.cut(TM1['Age'],custombins)
TM1['Edad3'].value_counts(sort=False)